This notebook is adapted from https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/13B_Visual_Analysis_MNIST.ipynb. It allows you to find the input 'image' which would produce the maximum activation for a given filter (or random selection of filters, or all filters) in a given layer of a loaded, pre-trained, U-Net.

In [23]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
from keras import layers, models
import numpy as np
import datetime
#import capsnet
from SegCaps import capsule_layers
import dataset
import model_functions as mf
import audio_functions as af
import audio_models

import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'hot'

In [29]:
#  Set variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=16
normalise=True
batch_size = 1
shuffle=False
n_shuffle = 1
mag_phase = True
learning_rate = 0.0002
model_variant = 'unet'
data_type = 'mag'
phase_weight = 0.0005

checkpoint = '234/234-8'
#model_base_dir = '/home/enterprise.internal.city.ac.uk/acvn728/checkpoints'
model_base_dir = 'C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints'

#directory_a = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Mixed'
#directory_b = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Voice'
#directory_c = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Background'

directory_a = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Mixed'
directory_b = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Voice'
directory_c = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Background'


#  Create the pipeline
tf.reset_default_graph()
data = dataset.zip_files(directory_a, directory_b, directory_c)
data = dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise)

#  Create the iterator
pipe = data.make_initializable_iterator()
mixed_spec, voice_spec, background_spec, mixed_audio, voice_audio, background_audio = pipe.get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_phase = tf.expand_dims(mixed_spec[:, :, :-1, 3], 3)
if data_type == 'mag':
    mixed_input = tf.expand_dims(mixed_spec[:, :, :-1, 2], 3)
    voice_input = tf.expand_dims(voice_spec[:, :, :-1, 2], 3)
elif data_type in ['mag_phase', 'mag_phase_diff']:
    mixed_input = mixed_spec[:, :, :-1, 2:4]
    voice_input = voice_spec[:, :, :-1, 2:4]
elif data_type == 'real_imag':
    mixed_input = mixed_spec[:, :, :-1, 0:2]
    voice_input = voice_spec[:, :, :-1, 0:2]
elif data_type == 'mag_real_imag':
    mixed_input = tf.concat([tf.expand_dims(mixed_spec[:, :, :-1, 2], 3), mixed_spec[:, :, :-1, 0:2]], 3)
    voice_input = tf.concat([tf.expand_dims(voice_spec[:, :, :-1, 2], 3), voice_spec[:, :, :-1, 0:2]], 3)
elif data_type == 'mag_phase_real_imag':
    mixed_input = mixed_spec[:, :, :-1, :]
    voice_input = voice_spec[:, :, :-1, :]
    
data

<PrefetchDataset shapes: ((?, 256, 513, 4), (?, 256, 513, 4), (?, 256, 513, 4), (?, 65280, 1), (?, 65280, 1), (?, 65280, 1)), types: (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>

In [30]:
# Build U-Net model
print('Creating model')
model = audio_models.MagnitudeModel(mixed_input, voice_input, mixed_phase, mixed_audio, 
                                    voice_audio, background_audio, model_variant, is_training, learning_rate, 
                                    data_type, phase_weight, name='Magnitude_Model')
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.visible_device_list = str(1)
sess = tf.Session(config=tf_config)
sess.run(tf.global_variables_initializer())

Creating model


In [31]:
# Load a saved checkpoint
print('Loading checkpoint')
checkpoint_path = os.path.join(model_base_dir, checkpoint)
restorer = tf.train.Saver()
restorer.restore(sess, checkpoint_path)

Loading checkpoint
INFO:tensorflow:Restoring parameters from C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints\234/234-8


In [73]:
def plot_images(images, smooth=True):
    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'
    
    num_cols = 4
    num_rows = int(len(images) / num_cols)
    
    # Create figure with sub-plots.
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(16,int(num_rows * 2)))

    # Adjust vertical spacing.
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # For each entry in the grid.
    for i, ax in enumerate(axes.flat):
        # Get the i'th image and only use the desired pixels.
        img = images[i, :, :]
        
        # Plot the image.
        ax.imshow(img, interpolation=interpolation)

        # Remove ticks.
        ax.set_xticks([])
        ax.set_yticks([])

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

Function for getting the names of all the convolutional layers in the neural network. We could have made this list manually, but for larger neural networks it is easier to do this with a function.

In [74]:
def get_conv_layer_names():
    graph = tf.get_default_graph()
    
    # Create a list of names for the operations in the graph
    # for the Inception model where the operator-type is 'Conv2D'.
    names = [op.name for op in graph.get_operations() if op.type=='Conv2D']

    return names

conv_names = get_conv_layer_names()

This function finds the input image that maximizes a given feature in the network. It essentially just performs optimization with gradient ascent. The image is initialized with small random values and is then iteratively updated using the gradient for the given feature with regard to the image.

In [75]:
def optimize_image(conv_id=0, feature=0,
                   num_iterations=30, show_progress=True):
    """
    Find an image that maximizes the feature
    given by the conv_id and feature number.

    Parameters:
    conv_id: Integer identifying the convolutional layer to
             maximize. It is an index into conv_names.
             If None then use the last fully-connected layer
             before the softmax output.
    feature: Index into the layer for the feature to maximize.
    num_iteration: Number of optimization iterations to perform.
    show_progress: Boolean whether to show the progress.
    """

    # Create the loss-function that must be maximized.

    # Get the name of the convolutional operator.
    conv_name = conv_names[conv_id]

    # Get the default TensorFlow graph.
    graph = tf.get_default_graph()

    # Get a reference to the tensor that is output by the
    # operator. Note that ":0" is added to the name for this.
    tensor = graph.get_tensor_by_name(conv_name + ":0")

    # The loss-function is the average of all the
    # tensor-values for the given feature. This
    # ensures that we generate the whole input image.
    # You can try and modify this so it only uses
    # a part of the tensor.
    loss = tf.reduce_mean(tensor[:,:,:,feature])

    # Get the gradient for the loss-function with regard to
    # the input image. This creates a mathematical
    # function for calculating the gradient.
    gradient = tf.gradients(loss, mixed_input)

    # Generate a random image of the same size as the raw input.
    # Each pixel is a small random value between 0.45 and 0.55,
    # which is the middle of the valid range between 0 and 1.
    img_shape = tuple(mixed_input.shape.as_list()[1:3])
    image = 0.1 * np.random.uniform(size=img_shape) + 0.45

    # Perform a number of optimization iterations to find
    # the image that maximizes the loss-function.
    for i in range(num_iterations):
        # Reshape the array so it is a 4-rank tensor.
        img_reshaped = image[np.newaxis,:,:,np.newaxis]

        # Create a feed-dict for inputting the image to the graph.
        feed_dict = {model.mixed_input: img_reshaped,
                     model.is_training: False}

        # Calculate the predicted class-scores,
        # as well as the gradient and the loss-value.
        grad, loss_value = sess.run([gradient, loss],
                                     feed_dict=feed_dict)
        
        # Squeeze the dimensionality for the gradient-array.
        grad = np.array(grad).squeeze()

        # The gradient now tells us how much we need to change the
        # input image in order to maximize the given feature.

        # Calculate the step-size for updating the image.
        # This step-size was found to give fast convergence.
        # The addition of 1e-8 is to protect from div-by-zero.
        step_size = 1.0 / (grad.std() + 1e-8)

        # Update the image by adding the scaled gradient
        # This is called gradient ascent.
        image += step_size * grad

        # Ensure all pixel-values in the image are between 0 and 1.
        image = np.clip(image, 0.0, 1.0)

        if show_progress:
            print("Iteration:", i)

            # Convert the predicted class-scores to a one-dim array.
            #pred = np.squeeze(pred)

            # The predicted class for the Inception model.
            #pred_cls = np.argmax(pred)

            # The score (probability) for the predicted class.
            #cls_score = pred[pred_cls]

            # Print the predicted score etc.
            #msg = "Predicted class: {0}, score: {1:>7.2%}"
            #print(msg.format(pred_cls, cls_score))

            # Print statistics for the gradient.
            msg = "Gradient min: {0:>9.6f}, max: {1:>9.6f}, stepsize: {2:>9.2f}"
            print(msg.format(grad.min(), grad.max(), step_size))

            # Print the loss-value.
            print("Loss:", loss_value)

            # Newline.
            print()

    return image.squeeze()

In [76]:
def optimize_images(conv_id=0, num_iterations=30, num_filters=None):
    """
    Find 10 images that maximize the 10 first features in the layer
    given by the conv_id.
    
    Parameters:
    conv_id: Integer identifying the convolutional layer to
             maximize. It is an index into conv_names.
             If None then use the last layer before the softmax output.
    num_iterations: Number of optimization iterations to perform.
    """

    # Which layer are we using?
    if conv_id is None:
        print("Final fully-connected layer before softmax.")
    else:
        print("Layer:", conv_names[conv_id])

    # Initialize the array of images.
    images = []
    
    # Get number of filters in layer
    conv_name = conv_names[conv_id]
    graph = tf.get_default_graph()
    tensor = graph.get_tensor_by_name(conv_name + ":0")
    num_filters_in_layer = tensor.shape[-1]
    
    # If number of filters not specified, assume all are required
    if num_filters is None:
        num_filters = num_filters_in_layer
        filters = np.arange(num_filters)
    else:
        # Randomly select which filters to use
        filters = np.random.randint(0, num_filters_in_layer, num_filters)
        
    # For each feature do the following.
    for conv_filter in filters:
        print("Optimizing image for filter no.", conv_filter)
        
        # Find the image that maximizes the given feature
        # for the network layer identified by conv_id (or None).
        image = optimize_image(conv_id=conv_id, feature=conv_filter,
                               show_progress=False,
                               num_iterations=num_iterations)

        # Squeeze the dim of the array.
        image = image.squeeze()

        # Append to the list of images.
        images.append(image)

    # Convert to numpy-array so we can index all dimensions easily.
    images = np.array(images)

    return images

In [80]:
images = optimize_images(conv_id=5, num_iterations=30)

Layer: Magnitude_Model/voice-mask-unet/encoder/layer-6/conv2d/Conv2D
Optimizing image for feature no. 0
Optimizing image for feature no. 1
Optimizing image for feature no. 2
Optimizing image for feature no. 3
Optimizing image for feature no. 4
Optimizing image for feature no. 5
Optimizing image for feature no. 6
Optimizing image for feature no. 7
Optimizing image for feature no. 8
Optimizing image for feature no. 9
Optimizing image for feature no. 10
Optimizing image for feature no. 11
Optimizing image for feature no. 12
Optimizing image for feature no. 13
Optimizing image for feature no. 14
Optimizing image for feature no. 15
Optimizing image for feature no. 16
Optimizing image for feature no. 17
Optimizing image for feature no. 18
Optimizing image for feature no. 19
Optimizing image for feature no. 20
Optimizing image for feature no. 21
Optimizing image for feature no. 22
Optimizing image for feature no. 23
Optimizing image for feature no. 24
Optimizing image for feature no. 25
Optim

Optimizing image for feature no. 223
Optimizing image for feature no. 224
Optimizing image for feature no. 225
Optimizing image for feature no. 226
Optimizing image for feature no. 227
Optimizing image for feature no. 228
Optimizing image for feature no. 229
Optimizing image for feature no. 230
Optimizing image for feature no. 231
Optimizing image for feature no. 232
Optimizing image for feature no. 233
Optimizing image for feature no. 234
Optimizing image for feature no. 235
Optimizing image for feature no. 236
Optimizing image for feature no. 237
Optimizing image for feature no. 238
Optimizing image for feature no. 239
Optimizing image for feature no. 240
Optimizing image for feature no. 241
Optimizing image for feature no. 242
Optimizing image for feature no. 243
Optimizing image for feature no. 244
Optimizing image for feature no. 245
Optimizing image for feature no. 246
Optimizing image for feature no. 247
Optimizing image for feature no. 248
Optimizing image for feature no. 249
O

Optimizing image for feature no. 445
Optimizing image for feature no. 446
Optimizing image for feature no. 447
Optimizing image for feature no. 448
Optimizing image for feature no. 449
Optimizing image for feature no. 450
Optimizing image for feature no. 451
Optimizing image for feature no. 452
Optimizing image for feature no. 453
Optimizing image for feature no. 454
Optimizing image for feature no. 455
Optimizing image for feature no. 456
Optimizing image for feature no. 457
Optimizing image for feature no. 458
Optimizing image for feature no. 459
Optimizing image for feature no. 460
Optimizing image for feature no. 461
Optimizing image for feature no. 462
Optimizing image for feature no. 463
Optimizing image for feature no. 464
Optimizing image for feature no. 465
Optimizing image for feature no. 466
Optimizing image for feature no. 467
Optimizing image for feature no. 468
Optimizing image for feature no. 469
Optimizing image for feature no. 470
Optimizing image for feature no. 471
O

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [28]:
    # Plot the images.
    plot_images(images=images)

['Magnitude_Model/voice-mask-unet/encoder/layer-1/conv2d/Conv2D',
 'Magnitude_Model/voice-mask-unet/encoder/layer-2/conv2d/Conv2D',
 'Magnitude_Model/voice-mask-unet/encoder/layer-3/conv2d/Conv2D',
 'Magnitude_Model/voice-mask-unet/encoder/layer-4/conv2d/Conv2D',
 'Magnitude_Model/voice-mask-unet/encoder/layer-5/conv2d/Conv2D',
 'Magnitude_Model/voice-mask-unet/encoder/layer-6/conv2d/Conv2D',
 'Magnitude_Model/gradients/Magnitude_Model/voice-mask-unet/decoder/layer-6/conv2d_transpose/conv2d_transpose_grad/Conv2D',
 'Magnitude_Model/gradients/Magnitude_Model/voice-mask-unet/decoder/layer-5/conv2d_transpose/conv2d_transpose_grad/Conv2D',
 'Magnitude_Model/gradients/Magnitude_Model/voice-mask-unet/decoder/layer-4/conv2d_transpose/conv2d_transpose_grad/Conv2D',
 'Magnitude_Model/gradients/Magnitude_Model/voice-mask-unet/decoder/layer-3/conv2d_transpose/conv2d_transpose_grad/Conv2D',
 'Magnitude_Model/gradients/Magnitude_Model/voice-mask-unet/decoder/layer-2/conv2d_transpose/conv2d_transpos

In [50]:
model.voice_mask_network.encoder.l1

<tf.Tensor 'Magnitude_Model/voice-mask-unet/encoder/layer-1/conv2d/BiasAdd:0' shape=(?, 128, 256, 16) dtype=float32>